In [4]:
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
#import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
#from livelossplot import PlotLossesTensorFlowKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


### plot sample images

In [ ]:
utils.datasets.fer.plot_example_images(plt).show()

In [ ]:
#look at image inbalance
for expression in os.listdir('../data-capstone/fer2013_2/train/'):
    print(str(len(os.listdir("train/" + expression))) + " " + expression + "images")

### generate training validation batches

In [5]:
img_size = 48
batch_size= 64

datagen_train = ImageDataGenerator(horizontal_flip =True) #data augmentation flip images randomly
#on horizontal axis
train_generator = datagen_train.flow_from_directory("../data-capstone/fer2013_2/train/", 
                                                   target_size = (img_size, img_size),#images will be resized if not 48X48
                                                   color_mode='grayscale',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle=True)

    
datagen_validation = ImageDataGenerator(horizontal_flip =True) #data augmentation flip images randomly
#on horizontal axis
validation_generator = datagen_validation.flow_from_directory("../data-capstone/fer2013_2/test/", 
                                                   target_size = (img_size, img_size),#images will be resized if not 48X48
                                                   color_mode='grayscale',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle=True)   

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# Build the model

In [6]:
model = Sequential()

#first conv
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization()) #works well with CNN a form of normalization
model.add(Activation('relu')) #nonlinearity to learn more complex functions
model.add(MaxPooling2D(pool_size=(2,2)))#shrinks the height and width by factor of 2
model.add(Dropout(0.25)) #prevents overfitting to the training data

#second conv
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

#3rd conv
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 


#conv layer
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(Dropout(0.25)) 

model.add(Dense(512))
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(Dropout(0.25)) 

model.add(Dense(7, activation='softmax'))


opt = Adam(lr=0.0005)
model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['Accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

C:\Users\eswan\anaconda3\envs\capstone\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


## Train and evaluate

In [7]:
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size  #calculate the number of steps
#the number of items in the training set floor division by the batch size
#we defined the batch_size earlier,
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, model='auto')


#callbacks = [PlotLossesTensorFlowKeras(), checkpoint, reduce_lr]
callbacks = [checkpoint, reduce_lr]


history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks


)


Epoch 1/15
 14/448 [..............................] - ETA: 19:42 - loss: 2.1194 - Accuracy: 0.1975

KeyboardInterrupt: 

## Save model

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
#camera.py

#detects face images, converts to grayscale, resizes, sends to pretrained model, gets predictions
#back and adds them to the video frame
import cv2
from model import FacialExpressionModel
import numpy as np

facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = FacialExpressionModel("model.json", "model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture("/home/rhyme/Desktop/Project/videos/facial_exp.mkv")
        #self.video = cv2.VideoCapture(0) #if you want your webcam

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        _, jpeg = cv2.imencode('.jpg', fr)
        return jpeg.tobytes()

In [ ]:
#main.py
#for interacting with flask
from flask import Flask, render_template, Response
from camera import VideoCamera


app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen(camera):
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


In [ ]:
#model.py
from tensorflow.keras.models import model_from_json
import numpy as np
import tensorflow as tf

#deals with memory allocation
config = tf.compat.v1.configProto()
config.gpu_options.per_process_gpu_memory_fraction=0.15
session = tf.compt.v1.Session(config=config)

class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

    def __init__(self, model_json_file, model_weights_file):
        with open(model_json_file, "r" as json_file:
            loaded_model_json = json_file.read()
            self.loaded = model_from_json(loaded_model_json)


        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model._make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]
    

In [ ]:
#index.html
<html>
  <head>
    <title>Facial Expression Recognition</title>
  </head>

  <body>
    <img id='bg' width=800px height= 640px scr="{{ url_for('video_feed') }}">
  </body>
  </html>